#### Определения

In [1]:
!pip install gensim
!pip install compress-fasttext

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for compress-fasttext: filename=compress_fasttext-0.1.3-py3-none-any.whl size=14583 sha256=da1c0c98ea17290d067b452d5be6ededf8fe5ba60ba565542d90051e28b2b0d8
  Stored in directory: /home/seb/.cache/pip/wheels/8d/cb/65/ebcb8ad616fbcb8a215c1a3632b7759acc3ebf256f2ad37a0f
Successfully built compress-fasttext

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [15]:
from gensim import models

In [16]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(x, y):
    return dot(x, y) / (norm(x) * norm(y))

print(cos_sim([1, 0, 0], (1, 0, 0)))
print(cos_sim([1, 0, 0], (0, 1, 0)))
print(cos_sim([1, 0, 0], (-1, 0, 0)))

1.0
0.0
-1.0


In [17]:
norm([4, 4, 4])

6.928203230275509

#### Обучим модель word2vec

In [30]:
sentences = []
with open('ru-text.txt') as fin:
    for line in fin:
        sentences.append(line.split(' '))

In [71]:
sentences

[['Интерне́т[1]',
  '(англ.',
  'Internet)',
  '—',
  'коммуникационная',
  'сеть',
  'и',
  'всемирная',
  'система',
  'объединённых',
  'компьютерных',
  'сетей',
  'для',
  'хранения',
  'и',
  'передачи',
  'информации[2].\n'],
 ['\n'],
 ['Раньше',
  'упоминался',
  'как',
  'Всемирная',
  'сеть',
  'и',
  'Глобальная',
  'сеть,',
  'а',
  'также',
  'просто',
  'Сеть.',
  'Построена',
  'на',
  'базе',
  'стека',
  'протоколов',
  'TCP/IP[3].',
  'На',
  'основе',
  'Интернета',
  'работает',
  'Всемирная',
  'паутина',
  '(World',
  'Wide',
  'Web,',
  'WWW)',
  'и',
  'множество',
  'других',
  'систем',
  'передачи',
  'данных.',
  'К',
  'началу',
  '2020',
  'года',
  'число',
  'пользователей',
  'достигло',
  '4,5',
  'млрд',
  'человек,',
  'что',
  'составляет',
  'более',
  '50',
  '%',
  'от',
  'всех',
  'жителей',
  'планеты',
  'Земля.',
  'Во',
  'многом',
  'это',
  'было',
  'обусловлено',
  'широким',
  'распространением',
  'сотовых',
  'сетей',
  'с',
  'досту

In [60]:

model = models.Word2Vec( min_count=1)
model.build_vocab(sentences)


In [61]:
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

(7816, 8800)

In [63]:
try:
    print(cos_sim(model.wv['в'], model.wv['к']))
    print(cos_sim(model.wv['доступ'], model.wv['хранения']))
    print(cos_sim(model.wv['интернет'], model.wv['интенет']))
except Exception as e:
    print(e)

0.08500861
0.10406948
"Key 'интенет' not present"


#### Обучим модель FastText

In [67]:
model = models.FastText(sentences, sg=1, vector_size=50,min_count=1)

In [68]:
try:
    print(cos_sim(model.wv['в'], model.wv['к']))
    print(cos_sim(model.wv['доступ'], model.wv['хранения']))
    print(cos_sim(model.wv['интернет'], model.wv['интенет']))
except Exception as e:
    print(e)

0.5055136
0.81045586
0.9614415


#### Загрузим оптимизированную модель FasText

In [69]:
import compress_fasttext
model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(
    'https://github.com/avidale/compress-fasttext/releases/download/gensim-4-draft/geowac_tokens_sg_300_5_2020-100K-20K-100.bin'
)

In [70]:
try:
    print(cos_sim(model['сеть'], model['интернет']))
    print(cos_sim(model['человек'], model['интернет']))
    print(cos_sim(model['интернет'], model['интенет']))
except Exception as e:
    print(e)

0.5231468445248223
0.279718340393375
0.6044784549643755


## Домашнее задание
1. Скачать датасет с фразами симпосонов - https://github.com/sujanjoejacob/Text-mining-with-Simpsons-Data
2. Обучить FastText/Word2vec на фразах персонажей
3. Визуализировать embeddings по самым частотным словам (top 1000)
4. Найти самые близкие слова для:
  homer - marge + bart
  bart - lisa + school
  marge - homer + home
5. Попробовать построить классификатор bart/lisa с использованием этих эмбеддингов